In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NC_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,3.711900e+14
1,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,3.706300e+14
2,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,3.718305e+14
3,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,3.716793e+14
4,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,3.702100e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,3.711900e+14
1,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,3.706300e+14
2,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,3.718305e+14
3,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,3.716793e+14
4,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,3.702100e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2168,1781417104,NC,"121 Nationwide Dr. Suite A, RALEIGH, NC",37.380333,-79.214934,NaN
2683,7382587207,NC,"5220 salibert ave, CHARLOTTE, NC",35.274214,-80.852204,NaN
5100,2121217102,NC,"1636 Hwy 64 E, ASHEBORO, NC",35.488200,-94.201285,NaN
6540,2613678702,NC,"2915 Hollywood Drive, Myrtle Beach, NC",33.720600,-78.879741,NaN
7898,9240967207,NC,"6685 US HWY 64, SPRING HOPE, NC",36.711496,-107.940177,NaN
8355,4630927710,NC,"3801 BEAM RD SUITE C, CHARLOTTE, NC",35.175406,-80.928578,NaN
8547,7986137006,NC,"749 US Hwy 74 Bus E, ROCKINGHAM, NC",34.939320,-79.773946,NaN
10117,1570367102,NC,"229 Thomas Drive, SHALLOTTE, NC",30.182192,-85.760806,NaN
10911,4594547302,NC,"5601 EXECUTIVE CENTER DRIVE SUITE 200, CHARLOT...",35.201513,-80.743684,NaN
11863,5839697109,NC,"1879 WHITE LAKE DR PMB 7286, ELIZABETHTOWN, NC",34.632305,-78.486871,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('37')] #NC
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/33 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

33


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10544/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2168,1781417104,NC,"121 Nationwide Dr. Suite A, RALEIGH, NC",37.380333,-79.214934,[]
2683,7382587207,NC,"5220 salibert ave, CHARLOTTE, NC",35.274214,-80.852204,[371190054062012]
5100,2121217102,NC,"1636 Hwy 64 E, ASHEBORO, NC",35.488200,-94.201285,[]
6540,2613678702,NC,"2915 Hollywood Drive, Myrtle Beach, NC",33.720600,-78.879741,[]
7898,9240967207,NC,"6685 US HWY 64, SPRING HOPE, NC",36.711496,-107.940177,[]
8355,4630927710,NC,"3801 BEAM RD SUITE C, CHARLOTTE, NC",35.175406,-80.928578,[371190059281026]
8547,7986137006,NC,"749 US Hwy 74 Bus E, ROCKINGHAM, NC",34.939320,-79.773946,[371539705002023]
10117,1570367102,NC,"229 Thomas Drive, SHALLOTTE, NC",30.182192,-85.760806,[]
10911,4594547302,NC,"5601 EXECUTIVE CENTER DRIVE SUITE 200, CHARLOT...",35.201513,-80.743684,[371190019183000]
11863,5839697109,NC,"1879 WHITE LAKE DR PMB 7286, ELIZABETHTOWN, NC",34.632305,-78.486871,[370179501022033]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10544/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2168,1781417104,NC,"121 Nationwide Dr. Suite A, RALEIGH, NC",37.380333,-79.214934,NaN
2683,7382587207,NC,"5220 salibert ave, CHARLOTTE, NC",35.274214,-80.852204,371190054062012
5100,2121217102,NC,"1636 Hwy 64 E, ASHEBORO, NC",35.488200,-94.201285,NaN
6540,2613678702,NC,"2915 Hollywood Drive, Myrtle Beach, NC",33.720600,-78.879741,NaN
7898,9240967207,NC,"6685 US HWY 64, SPRING HOPE, NC",36.711496,-107.940177,NaN
8355,4630927710,NC,"3801 BEAM RD SUITE C, CHARLOTTE, NC",35.175406,-80.928578,371190059281026
8547,7986137006,NC,"749 US Hwy 74 Bus E, ROCKINGHAM, NC",34.939320,-79.773946,371539705002023
10117,1570367102,NC,"229 Thomas Drive, SHALLOTTE, NC",30.182192,-85.760806,NaN
10911,4594547302,NC,"5601 EXECUTIVE CENTER DRIVE SUITE 200, CHARLOT...",35.201513,-80.743684,371190019183000
11863,5839697109,NC,"1879 WHITE LAKE DR PMB 7286, ELIZABETHTOWN, NC",34.632305,-78.486871,370179501022033


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2168,1781417104,NC,"121 Nationwide Dr. Suite A, RALEIGH, NC",37.380333,-79.214934,NaN
5100,2121217102,NC,"1636 Hwy 64 E, ASHEBORO, NC",35.488200,-94.201285,NaN
6540,2613678702,NC,"2915 Hollywood Drive, Myrtle Beach, NC",33.720600,-78.879741,NaN
7898,9240967207,NC,"6685 US HWY 64, SPRING HOPE, NC",36.711496,-107.940177,NaN
10117,1570367102,NC,"229 Thomas Drive, SHALLOTTE, NC",30.182192,-85.760806,NaN
12031,4412518409,NC,"3090 E Highway 27, Lincolnton, NC",27.642438,-81.523059,NaN
12398,6946807203,NC,"3090 E HIGHWAY 27, LINCOLNTON, NC",27.642438,-81.523059,NaN
13829,4105847102,NC,"738 Hwy 27 E, Albemarle, NC",27.128900,-81.330759,NaN
14093,2434337104,NC,"3600 CHOCTAW RD, MATTHEWS, NC",37.534391,-77.558628,NaN
14669,1645817101,NC,"7453 US Highway 74 W, Polkton, NC",35.029108,-84.379122,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
2168,1781417104,NaN
2683,7382587207,371190054062012
5100,2121217102,NaN
6540,2613678702,NaN
7898,9240967207,NaN
8355,4630927710,371190059281026
8547,7986137006,371539705002023
10117,1570367102,NaN
10911,4594547302,371190019183000
11863,5839697109,370179501022033


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NC_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,521441,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,3.711900e+14
1,521442,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,3.706300e+14
2,521443,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,3.718305e+14
3,521444,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,3.716793e+14
4,521445,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,3.702100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,521441,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,3.711900e+14,NaN
1,521442,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,3.706300e+14,NaN
2,521443,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,3.718305e+14,NaN
3,521444,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,3.716793e+14,NaN
4,521445,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,3.702100e+14,NaN
...,...,...,...,...,...,...,...,...
22791,544232,9436857004,NC,"140 SANDERS ST, HOPE MILLS, NC",34.957494,-78.919675,3.705100e+14,NaN
22792,544233,9518447003,NC,"694 N Main Street, TROUTMAN, NC",35.717693,-80.897008,3.709706e+14,NaN
22793,544234,9747658410,NC,"6675 Falls of Neuse Rd Ste 017, Raleigh, NC",35.874854,-78.624424,3.718305e+14,NaN
22794,544235,9842517808,NC,"140 TENNESSEE CIR, MOORESVILLE, NC",35.648847,-80.901805,3.709706e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,521441,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,3.711900e+14,NaN,371190031111004.0
1,521442,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,3.706300e+14,NaN,370630017095008.0
2,521443,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,3.718305e+14,NaN,371830504001019.0
3,521444,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,3.716793e+14,NaN,371679312032005.0
4,521445,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,3.702100e+14,NaN,370210022044005.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2065677410,NC,"4601 Park Road Suite 250, CHARLOTTE, NC",35.168576,-80.850099,371190031111004.0
1,2101437308,NC,"120 WILLIAM PENN PLZ, DURHAM, NC",36.042449,-78.898323,370630017095008.0
2,3760207209,NC,"500 W Peace Street, Raleigh, NC",35.788949,-78.645503,371830504001019.0
3,3810107300,NC,"350 PEE DEE AVE STE A, ALBEMARLE, NC",35.350635,-80.193720,371679312032005.0
4,4697357307,NC,"1978 Hendersonville Road, ASHEVILLE, NC",35.489793,-82.523361,370210022044005.0
...,...,...,...,...,...,...
22791,9436857004,NC,"140 SANDERS ST, HOPE MILLS, NC",34.957494,-78.919675,370510030031020.0
22792,9518447003,NC,"694 N Main Street, TROUTMAN, NC",35.717693,-80.897008,370970612013052.0
22793,9747658410,NC,"6675 Falls of Neuse Rd Ste 017, Raleigh, NC",35.874854,-78.624424,371830540061006.0
22794,9842517808,NC,"140 TENNESSEE CIR, MOORESVILLE, NC",35.648847,-80.901805,370970612041004.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2168,1781417104,NC,"121 Nationwide Dr. Suite A, RALEIGH, NC",37.380333,-79.214934,NaN
5100,2121217102,NC,"1636 Hwy 64 E, ASHEBORO, NC",35.488200,-94.201285,NaN
6540,2613678702,NC,"2915 Hollywood Drive, Myrtle Beach, NC",33.720600,-78.879741,NaN
7898,9240967207,NC,"6685 US HWY 64, SPRING HOPE, NC",36.711496,-107.940177,NaN
10117,1570367102,NC,"229 Thomas Drive, SHALLOTTE, NC",30.182192,-85.760806,NaN
12031,4412518409,NC,"3090 E Highway 27, Lincolnton, NC",27.642438,-81.523059,NaN
12398,6946807203,NC,"3090 E HIGHWAY 27, LINCOLNTON, NC",27.642438,-81.523059,NaN
13829,4105847102,NC,"738 Hwy 27 E, Albemarle, NC",27.128900,-81.330759,NaN
14093,2434337104,NC,"3600 CHOCTAW RD, MATTHEWS, NC",37.534391,-77.558628,NaN
14669,1645817101,NC,"7453 US Highway 74 W, Polkton, NC",35.029108,-84.379122,NaN


Convert to File

In [19]:
fips_NC = fips_merge_drop

In [20]:
fips_NC.to_csv("../../../data/state_data/geo/geo_fips/23/NC_fips_scraped.csv") 